In [1]:
class Args:
    def __init__(self):
        self.controller = ['longformer_classifier', 'longformer_classifier']  # replace with your choice
        self.controller_model_string = ['allenai/longformer-base-4096', 'allenai/longformer-base-4096']
        self.loader = ['alignment', 'coherence']
        self.controller_save_dir = None
        self.controller_load_dir = ['doc_data/ckpt/coherence_reranker', 'doc_data/ckpt/relevance_reranker']
        self.controller_epochs = 1
        self.control_strength = [1.0]
        self.fudge_top_k = [100]
        self.fudge_batch_size = 256
        self.controller_num_negatives = 1
        self.coherence_negative_categories = ['other', 'repeat', 'shuffle']
        self.controller_lr = 5e-5

args = Args()


In [2]:
from story_generation.common.controller.models.longformer_classifier import LongformerClassifier

In [3]:
relevance_controller = LongformerClassifier(args, 0)

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/mbondarenko/anaconda3/envs/doc-story-generation/lib/python3.8/site-packages/transformers/optimization.py:429: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [5]:
coherence_controller = LongformerClassifier(args, 1)

Some weights of LongformerForSequenceClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.out_proj.bias', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
completions = ["""Aimee felt a sudden sense of panic and fear. She had the feeling that someone was watching her, that someone or something was waiting for her to be completely vulnerable before striking out at her. She found herself looking around her apartment, trying to see something or someone who could be causing this feeling. But she found nothing there, only the light from the lamp moving gently against her walls and ceiling. Silence was all around her, and it seemed so absolute that she felt as if she were being smothered by its weight. Silence was all-encompassing. In her apartment.

Silence was all-encompassing in her own small corner of the world, and it was suffocating to her. Silence pressed in on her from all sides, making it hard for her to breathe, and she felt suddenly overwhelmed by it.  She started to cry.

She cried because the silence was terrifying. She cried because it was so loud, so overwhelming. She cried because she felt alone in a way that no one should have to feel alone, her own small safe island surrounded by an ocean of noise and voices that had suddenly turned into nothing more than a silent and lonely roaring emptiness.

She felt exhausted and drained, drained of all the energy it took to simply get through her day. She felt lonely, unable to see past the walls that surrounded her, unable to even reach out for something solid with which she could hold on. She felt defeated by the  silence around her because she knew that it would never be silent again, at least not for a very long time.

She cried because there was no one in her apartment, there was no where else to go, there was nothing else to do but cry and wonder why the hell she had ever come back in the first place. She cried because she suddenly felt so alone, more alone than even being at work with all those people who were unable to see past her mask, and all she wanted was to be surrounded by voices and noise.

She kept crying as she washed her face and brushed her teeth, crying because there was""", """HuggingFace, you need to be authorized. This typically means you need to pass a valid token that has permissions to access that repository. You can log in via the HuggingFace CLI tool using huggingface-cli login and then use the generated token in your script. If you're already logged in, ensure your token has the correct permissions.

Internet Connection and URL Access: Make sure your internet connection is stable and that you're able to access the HuggingFace Hub URL (https://huggingface.co/). Sometimes, network issues or firewalls can block access to certain URLs."""]

previous = ["""Aimee Kincaid came home to an empty apartment. She found the silence deafening. She felt alone, even though she had spent the day around people, listening to them talk and laugh and complain. She knew how it sounded when a person sat in solitude after a long trip. She would return from a long journey and sit on her bed and watch the light through the blinds or listen to her neighbor’s conversations with his wife on their balcony.

But she had never been home alone like this. Maybe that was why she felt so strange. So different, out of place. The solitude was new to her.

She went into the bathroom and turned on the shower and then, without really knowing why she wanted to, she moved through the apartment turning on all the lights. She turned them on in the kitchen, in her bedroom, even in the closet where nothing but old clothes were kept neatly folded. She  lit candles strategically placed around the room so that shadows could not obscure her vision or movement. She sat on her couch and listened to the silence, stared at the flickering lights, waiting for them to hypnotize her into a dreamy state of mind.

She was very tired but could not sleep. She stood up and found her way to the bedroom where she stripped off her clothes, put on a nightgown, and climbed into bed. She turned off all the lights except for the bedside lamp and lay there listening to the ticking of the wall clock and to the faint drone of cars passing on the street below. Silence was the only sound in the apartment now, and it was completely deafening.""", """Aimee Kincaid came home to an empty apartment. She found the silence deafening. She felt alone, even though she had spent the day around people, listening to them talk and laugh and complain. She knew how it sounded when a person sat in solitude after a long trip. She would return from a long journey and sit on her bed and watch the light through the blinds or listen to her neighbor’s conversations with his wife on their balcony.

But she had never been home alone like this. Maybe that was why she felt so strange. So different, out of place. The solitude was new to her.

She went into the bathroom and turned on the shower and then, without really knowing why she wanted to, she moved through the apartment turning on all the lights. She turned them on in the kitchen, in her bedroom, even in the closet where nothing but old clothes were kept neatly folded. She  lit candles strategically placed around the room so that shadows could not obscure her vision or movement. She sat on her couch and listened to the silence, stared at the flickering lights, waiting for them to hypnotize her into a dreamy state of mind.

She was very tired but could not sleep. She stood up and found her way to the bedroom where she stripped off her clothes, put on a nightgown, and climbed into bed. She turned off all the lights except for the bedside lamp and lay there listening to the ticking of the wall clock and to the faint drone of cars passing on the street below. Silence was the only sound in the apartment now, and it was completely deafening."""]

In [7]:
coherence_scores = coherence_controller(previous, completions).cpu().numpy()
# logprob for alignment with previous story, up to 1k prev tokens

In [8]:
coherence_scores

array([-0.88256663, -0.8566264 ], dtype=float32)

In [ ]:
    def calculate_alignment(self, completions, prompt, node):
        if self.args.max_candidates == 1:
            return np.zeros(len(completions)) # in this case, we're doing no reranking, and this will also prevent the reranking from being used to decide when to stop. 
        unstripped_completions = completions
        completions = [c.strip() for c in completions]
        repetition_penalty = np.array([calculate_repetition_length_penalty(c, [prompt]) for c in completions])
        last_prompt_paragraph = split_paragraphs(prompt, mode='newline')[-1]
        is_first_person = np.array([1 if detect_first_second_person(last_prompt_paragraph + c) - detect_first_second_person(last_prompt_paragraph) else 0 for c in completions]) # could have some false positives if the quotations are off, but whatever.
        repetition_penalty += is_first_person
        alignment_score = 0

        # for relevance, since we generate shorter passages at a time before refreshing the prompt compared to re3, 
        # we'll compute the min of two relevance scores for slightly better robustness. 
        # specifically we align against 
        # (1) the text generated thus far for this outline node, and 
        # (2) the text generated thus far for the outline node, prefixed by the last generated part of the preceding outline node (if it exists)
        if self.args.controller[0] == 'longformer_classifier':
            previous_outline_section = self.detailed_outline_section_history[-2] if len(self.detailed_outline_section_history) > 1 else None
            if previous_outline_section is not None and len(self.paragraphs_by_outline_section[previous_outline_section]) > 0:
                previous_text = self.paragraphs_by_outline_section[previous_outline_section][-1]
            else:
                previous_text = ''
            alignment_input = [create_prefix_completion(''.join(self.paragraphs_by_outline_section[node]) + c, node.text)[1] for c in unstripped_completions]
            prefix_alignment_input = [create_prefix_completion(previous_text + ''.join(self.paragraphs_by_outline_section[node]) + c, node.text)[1] for c in unstripped_completions]
            logging.log(22, 'prefix alignment input 0: ' + str(prefix_alignment_input[0]))
        else:
            raise NotImplementedError
        relevance_scores = self.controllers[0].evaluate_overall_texts(alignment_input).cpu().numpy() # logprob for alignment with outline
        logging.log(22, 'relevance scores: ' + str(['%.2f' % score for score in relevance_scores]))
        prefix_relevance_scores = self.controllers[0].evaluate_overall_texts(prefix_alignment_input).cpu().numpy()
        logging.log(22, 'prefix relevance scores: ' + str(['%.2f' % score for score in prefix_relevance_scores]))
        relevance_scores = np.array([min(rs, prs) for rs, prs in zip(relevance_scores, prefix_relevance_scores)]) # take the minimum of the two for robustness, to avoid occasional fake signal from where the continuation got cut off
        logging.log(22, 'min relevance scores: ' + str(['%.2f' % score for score in relevance_scores]))
        alignment_score += relevance_scores * self.args.control_strength[0]

        # similar relevance checking for scenes/characters
        current_node = node
        if len(self.detailed_outline_section_history) > 1:
            previous_node = self.detailed_outline_section_history[-2]
            extra_relevance_strings = []
            if not is_same_scene(current_node.scene, previous_node.scene):
                extra_relevance_strings.append(('The characters move to ' + current_node.scene, self.args.control_strength[0] * 0.5))
            for character in current_node.selected_entities:
                if character not in previous_node.selected_entities:
                    extra_relevance_strings.append((character + ' enters the scene.', self.args.control_strength[0] * 0.2))
            for ers, cs in extra_relevance_strings:
                logging.log(22, 'scene/char relevance string: ' + ers)
                extra_alignment_input = [create_prefix_completion(''.join(self.paragraphs_by_outline_section[node]) + c, ers)[1] for c in unstripped_completions]
                extra_prefix_alignment_input = [create_prefix_completion(previous_text + ''.join(self.paragraphs_by_outline_section[node]) + c, ers)[1] for c in unstripped_completions]
                extra_relevance_scores = self.controllers[0].evaluate_overall_texts(extra_alignment_input).cpu().numpy()
                logging.log(22, 'scene/char relevance scores: ' + str(['%.2f' % score for score in extra_relevance_scores]))
                extra_prefix_relevance_scores = self.controllers[0].evaluate_overall_texts(extra_prefix_alignment_input).cpu().numpy()
                logging.log(22, 'scene/char prefix relevance scores: ' + str(['%.2f' % score for score in extra_prefix_relevance_scores]))
                extra_relevance_scores = np.array([min(rs, prs) for rs, prs in zip(extra_relevance_scores, extra_prefix_relevance_scores)])
                logging.log(22, 'min scene/char relevance scores: ' + str(['%.2f' % score for score in extra_relevance_scores]))
                alignment_score += extra_relevance_scores * cs

        # coherence reranker
        if len(self.story().strip()) > 0:
            coherence_scores = self.controllers[1]([self.previous_passage(1000) for _ in range(len(completions))], completions).cpu().numpy() # logprob for alignment with previous story, up to 1k prev tokens
            logging.log(22, 'coherence scores: ' + str(['%.2f' % score for score in coherence_scores]))
            alignment_score += coherence_scores * self.args.control_strength[1]
        else:
            alignment_score += -1 * self.args.control_strength[1] # add some baseline level to prevent early stopping

        # heuristics e.g. repetition penalty
        logging.log(22, 'repetition: ' + str(['%.2f' % score for score in -repetition_penalty * self.args.repetition_penalty_weight]))
        alignment_score += -repetition_penalty * self.args.repetition_penalty_weight
        return alignment_score